<a href="https://colab.research.google.com/github/rudysemola/AutoCL-materials/blob/main/FLAML_tune_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
NOTE: only HPO (tune) part:
1. Tuning obj (UDF)
2. Search space
3. Constraints (ex. time budget)

This notebook includes simple examples to demonstrate how to tune User Defined Functions with `flaml.tune`.

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with options:
```bash
pip install flaml[notebook]
```

In [ ]:
%pip install flaml[notebook]

# Basic tuning procedure
## 1. A basic tuning example

In [ ]:
'''Set a search space'''
from flaml import tune

config_search_space = {
    "x": tune.lograndint(lower=1, upper=100000),
    "y": tune.randint(lower=1, upper=100000)
}  

In [ ]:
'''Write a evaluation function'''
import time

def evaluate_config(config: dict):
    """evaluate a hyperparameter configuration"""
    score = (config["x"] - 85000) ** 2 - config["x"] / config["y"]
    # usually the evaluation takes an non-neglible cost
    # and the cost could be related to certain hyperparameters
    # here we simulate this cost by calling the time.sleep() function
    # here we assume the cost is proportional to x
    faked_evaluation_cost = config["x"] / 100000
    time.sleep(faked_evaluation_cost)
    # we can return a single float as a score on the input config:
    # return score
    # or, we can return a dictionary that maps metric name to metric value:
    return {"score": score, "evaluation_cost": faked_evaluation_cost, "constraint_metric": config["x"] * config["y"]}

In [ ]:
'''Performs tuning'''
# require: pip install flaml[blendsearch]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
)

[flaml.tune.tune: 03-02 15:43:53] {528} WARNING - Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 1 config: {'x': 3, 'y': 13184}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 2 config: {'x': 2, 'y': 372}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 3 config: {'x': 6, 'y': 25996}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 4 config: {'x': 17, 'y': 15109}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 5 config: {'x': 33, 'y': 2144}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 6 config: {'x': 11, 'y': 1}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 7 config: {'x': 97, 'y': 12773}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 8 config: {'x': 494, 'y': 12615}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 9 config: {'x': 254, 'y': 25529}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 10 config: {'x': 959, 'y': 1}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 11 config: {'x': 4285, 'y': 1}
[flaml.tune.tune: 03-02 15:43:54] {811} INFO - trial 12 config: {'x': 959, '

In [ ]:
'''Investigate results'''
print(analysis.best_result)

{'score': 224870002.0, 'evaluation_cost': 0.99999, 'constraint_metric': 99999, 'training_iteration': 0, 'config': {'x': 99999, 'y': 1}, 'config/x': 99999, 'config/y': 1, 'experiment_tag': 'exp', 'time_total_s': 1.0021584033966064}


## Hierarchical search space 
Hierarchical search space is supported.

In [ ]:
'''Set a hierarchical search space'''
gbtree_hp_space = {
    "booster": "gbtree",
    "n_estimators": tune.lograndint(lower=4, upper=64),
    "max_leaves": tune.lograndint(lower=4, upper=64),
    "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
}
gblinear_hp_space = {
    "booster": "gblinear",
    "lambda": tune.uniform(0, 1),
    "alpha": tune.loguniform(0.0001, 1),
}

full_space = {
    "xgb_config": tune.choice([gbtree_hp_space, gblinear_hp_space]),
}

In [ ]:
'''Write a evaluation function'''
import xgboost as xgb

def xgb_obj(X_train, X_test, y_train, y_test, config):
    config = config["xgb_config"]
    params = config2params(config)
    dtrain = xgb.DMatrix(X_train, label=y_train)
    booster_type = config.get("booster")

    if booster_type == "gblinear":
        model = xgb.train(
            params,
            dtrain,
        )
    else:
        _n_estimators = params.pop("n_estimators")
        model = xgb.train(params, dtrain, _n_estimators)

    # get validation loss
    from sklearn.metrics import r2_score

    dtest = xgb.DMatrix(X_test)
    y_test_predict = model.predict(dtest)
    test_loss = 1.0 - r2_score(y_test, y_test_predict)
    return {"loss": test_loss}

def config2params(config: dict) -> dict:
    params = config.copy()
    max_depth = params["max_depth"] = params.get("max_depth", 0)
    if max_depth == 0:
        params["grow_policy"] = params.get("grow_policy", "lossguide")
        params["tree_method"] = params.get("tree_method", "hist")
    # params["booster"] = params.get("booster", "gbtree")
    params["use_label_encoder"] = params.get("use_label_encoder", False)
    if "n_jobs" in config:
        params["nthread"] = params.pop("n_jobs")
    return params

In [ ]:
'''Tune xgb_obj with configs from the hierarchical search space'''
from flaml.data import load_openml_dataset
from functools import partial

X_train, X_test, y_train, y_test = load_openml_dataset(
    dataset_id=537, data_dir="./"
)
analysis = tune.run(
    partial(xgb_obj, X_train, X_test, y_train, y_test),
    config=full_space,
    metric="loss",
    mode="min",
    num_samples=5,
)
print("analysis", analysis.results)

download dataset from openml


DEBUG:openml.datasets.dataset:Saved dataset 537: houses to file /root/.openml/cache/org/openml/www/datasets/537/dataset.pkl.py3


Dataset name: houses
X_train.shape: (15480, 8), y_train.shape: (15480,);
X_test.shape: (5160, 8), y_test.shape: (5160,)
[flaml.tune.tune: 03-02 15:45:08] {528} WARNING - Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.tune.tune: 03-02 15:45:08] {811} INFO - trial 1 config: {'xgb_config': {'booster': 'gblinear', 'lambda': 0.6472660813321921, 'alpha': 0.0028264214081400044}}
[15:45:09] WARNING: ../src/learner.cc:767: 
Parameters: { "grow_policy", "max_depth", "tree_method", "use_label_encoder" } are not used.

[flaml.tune.tune: 03-02 15:45:09] {811} INFO - trial 2 config: {'xgb_config': {'booster': 'gblinear', 'lambda': 0.5873922119376107, 'alpha': 0.0008684621556932634}}
[15:45:09] WARNING: ../src/learner.cc:767: 
Parameters: { "grow_policy", "max_depth", "tree_method", "use_label_encoder" } are not used.

[flaml.tune.tune: 03-02 15:45:09] {811} INFO - trial 3 config: {'xgb_config': {'booster': 'gblinear', 'lambda': 0.6776471695756079, 'alpha': 0.0003186000282430812}}
[15:45:09] WARNING: ../src/learner.cc:767: 
Parameters: { "grow_policy", "max_depth", "tree_method", "use_label_encoder" } are not used.

[flaml.tune.tune: 03-02 15:45:09] {811} INFO - trial 4 config: {'xgb_config': {'booster': '

# Advanced Tuning Options

## Parallel tuning

In [ ]:
%pip install flaml[ray]

In [ ]:
# require: pip install flaml[ray]
analysis = tune.run(
    evaluate_config,  # the function to evaluate a config
    config=config_search_space,  # the search space defined
    metric="score",
    mode="min",  # the optimization mode, "min" or "max"
    num_samples=-1,  # the maximal number of configs to try, -1 means infinite
    time_budget_s=10,  # the time budget in seconds
    use_ray=True,
    resources_per_trial={"cpu": 2}  # limit resources allocated per trial
)
print(analysis.best_trial.last_result)  # the best trial's result
print(analysis.best_config)  # the best config

Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]
INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


AttributeError: ignored

## Warm start

In [ ]:
config_search_space = {
    "a": tune.uniform(lower=0, upper=0.99),
    "b": tune.uniform(lower=0, upper=3),
}

def simple_obj(config):
    return config["a"] + config["b"]

points_to_evaluate = [
    {"b": .99, "a": 3},
    {"b": .99, "a": 2},
    {"b": .80, "a": 3},
    {"b": .80, "a": 2},
]
evaluated_rewards = [3.99, 2.99]

analysis = tune.run(
    simple_obj,
    config=config_search_space,
    mode="max",
    points_to_evaluate=points_to_evaluate,
    evaluated_rewards=evaluated_rewards,
    num_samples=10,
)

[flaml.tune.tune: 03-02 15:48:21] {528} WARNING - Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.tune.tune: 03-02 15:48:21] {811} INFO - trial 1 config: {'b': 0.8, 'a': 3.0}
[flaml.tune.tune: 03-02 15:48:21] {811} INFO - trial 2 config: {'b': 0.8, 'a': 2.0}


AttributeError: ignored

## Trial scheduling

###  An authentic scheduler implemented in FLAML (`scheduler='flaml'`).

In [ ]:
search_space = {
    "n_estimators": tune.lograndint(lower=4, upper=32768),
    "num_leaves": tune.lograndint(lower=4, upper=32768),
    "learning_rate": tune.loguniform(lower=1 / 1024, upper=1.0),
}

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

'''Set a evaluation function with resource dimension'''
def obj_from_resource_attr(resource_attr, X_train, X_test, y_train, y_test, config):

    # in this example sample size is our resource dimension
    resource = int(config[resource_attr])
    sampled_X_train = X_train.iloc[:resource]
    sampled_y_train = y_train.iloc[:resource]

    # construct a LGBM model from the config
    # note that you need to first remove the resource_attr field
    # from the config as it is not part of the original search space
    model_config = config.copy()
    del model_config[resource_attr]
    model = LGBMClassifier(**model_config)

    model.fit(sampled_X_train, sampled_y_train)
    y_test_predict = model.predict(X_test)
    test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
    return {"loss": test_loss}

In [ ]:
from flaml.data import load_openml_task
from sklearn.utils import shuffle

X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
# shuffle X_train and y_train
X_train, y_train = shuffle(X_train, y_train)
max_resource = len(y_train)
resource_attr = "sample_size"
min_resource = 10000
analysis = tune.run(
    partial(
        obj_from_resource_attr, resource_attr, X_train, X_test, y_train, y_test
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="flaml",
    max_resource=max_resource,
    min_resource=min_resource,
    time_budget_s=300,
    num_samples=-1,
)
print("best result w/ flaml scheduler (in 300s): ", analysis.best_result)

DEBUG:openml.datasets.dataset:Saved dataset 1590: adult to file /root/.openml/cache/org/openml/www/datasets/1590/dataset.pkl.py3
DEBUG:openml.datasets.dataset:Data pickle file already exists and is up to date.


download dataset from openml
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)
[flaml.tune.tune: 03-02 15:49:25] {528} WARNING - Using CFO for search. To use BlendSearch, run: pip install flaml[blendsearch]


INFO:flaml.tune.searcher.blendsearch:No low-cost partial config given to the search algorithm. For cost-frugal search, consider providing low-cost values for cost-related hps via 'low_cost_partial_config'. More info can be found at https://microsoft.github.io/FLAML/docs/FAQ#about-low_cost_partial_config-in-tune


[flaml.tune.tune: 03-02 15:49:25] {811} INFO - trial 1 config: {'n_estimators': 9, 'num_leaves': 1364, 'learning_rate': 0.012074374674294664, 'sample_size': 10000}
[flaml.tune.tune: 03-02 15:49:25] {811} INFO - trial 2 config: {'n_estimators': 8, 'num_leaves': 1156, 'learning_rate': 0.03978162762775204, 'sample_size': 10000}
[flaml.tune.tune: 03-02 15:49:25] {811} INFO - trial 3 config: {'n_estimators': 10, 'num_leaves': 1609, 'learning_rate': 0.003664770208485474, 'sample_size': 10000}
[flaml.tune.tune: 03-02 15:49:26] {811} INFO - trial 4 config: {'n_estimators': 4, 'num_leaves': 2287, 'learning_rate': 0.004845793654073492, 'sample_size': 10000}
[flaml.tune.tune: 03-02 15:49:26] {811} INFO - trial 5 config: {'n_estimators': 22, 'num_leaves': 813, 'learning_rate': 0.030085995026365456, 'sample_size': 10000}
[flaml.tune.tune: 03-02 15:49:26] {811} INFO - trial 6 config: {'n_estimators': 17, 'num_leaves': 595, 'learning_rate': 0.0956948042435157, 'sample_size': 10000}
[flaml.tune.tune: 

###  ASHA scheduler (`scheduler='asha'`) or a custom scheduler of the  [`TrialScheduler`](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers) class from `ray.tune`.

In [ ]:
def obj_w_intermediate_report(
    resource_attr,
    X_train,
    X_test,
    y_train,
    y_test,
    min_resource,
    max_resource,
    config,
):
    # a customized schedule to perform the evaluation
    eval_schedule = [res for res in range(min_resource, max_resource, 5000)] + [
        max_resource
    ]
    for resource in eval_schedule:
        sampled_X_train = X_train.iloc[:resource]
        sampled_y_train = y_train.iloc[:resource]

        # construct a LGBM model from the config
        model = LGBMClassifier(**config)

        model.fit(sampled_X_train, sampled_y_train)
        y_test_predict = model.predict(X_test)
        test_loss = 1.0 - accuracy_score(y_test, y_test_predict)
        # need to report the resource attribute used and the corresponding intermediate results
        try:
            tune.report(sample_size=resource, loss=test_loss)
        except StopIteration:
            return

In [ ]:
X_train, X_test, y_train, y_test = load_openml_task(task_id=7592, data_dir="")
resource_attr = "sample_size"
min_resource = 10000
max_resource = len(y_train)
analysis = tune.run(
    partial(
        obj_w_intermediate_report,
        resource_attr,
        X_train,
        X_test,
        y_train,
        y_test,
        min_resource,
        max_resource,
    ),
    config=search_space,
    metric="loss",
    mode="min",
    resource_attr=resource_attr,
    scheduler="asha",
    max_resource=max_resource,
    min_resource=min_resource,
    time_budget_s=300,
    num_samples=-1,
)
print("best result w/ asha scheduler (in 300s): ", analysis.best_result)

[flaml.tune.tune: 01-09 06:20:34] {486} INFO - Using search algorithm type.
2023-01-09 06:20:34,387	WARNING optuna.py:297 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `config` argument to `tune.run()` instead.
[I 2023-01-09 06:20:34,388] A new study created in memory with name: optuna
[flaml.tune.tune: 01-09 06:20:34] {636} INFO - trial 1 config: {'n_estimators': 9, 'num_leaves': 1364, 'learning_rate': 0.012074374674294664}


load dataset from openml_task7592.pkl
X_train.shape: (43957, 14), y_train.shape: (43957,),
X_test.shape: (4885, 14), y_test.shape: (4885,)


[flaml.tune.tune: 01-09 06:20:36] {636} INFO - trial 2 config: {'n_estimators': 4048, 'num_leaves': 4, 'learning_rate': 0.07891713267442702}
[flaml.tune.tune: 01-09 06:21:11] {636} INFO - trial 3 config: {'n_estimators': 3295, 'num_leaves': 334, 'learning_rate': 0.004638797085780012}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 4 config: {'n_estimators': 21, 'num_leaves': 3668, 'learning_rate': 0.003153366048206083}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 5 config: {'n_estimators': 8, 'num_leaves': 1845, 'learning_rate': 0.7239356970260848}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 6 config: {'n_estimators': 4, 'num_leaves': 379, 'learning_rate': 0.2728556109672425}
[flaml.tune.tune: 01-09 06:24:39] {636} INFO - trial 7 config: {'n_estimators': 948, 'num_leaves': 2573, 'learning_rate': 0.0073847289359894605}


best result w/ asha scheduler (in 300s):  {'sample_size': 43957, 'loss': 0.12302968270214942, 'training_iteration': 7, 'config': {'n_estimators': 4048, 'num_leaves': 4, 'learning_rate': 0.07891713267442702}, 'config/n_estimators': 4048, 'config/num_leaves': 4, 'config/learning_rate': 0.07891713267442702, 'experiment_tag': 'exp', 'time_total_s': 35.0105664730072}
